In [15]:
import cv2
import os
from mtcnn import MTCNN

# Step 1: Read the video
video_path = 'Videos/gvasarkpfh.mp4'
# video_path = 'Videos/ajeegjgyzk.mp4'
cap = cv2.VideoCapture(video_path)

# Create a folder to save cropped images
output_folder = 'cropped_images'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Initialize MTCNN detector
detector = MTCNN()

# Step 2: Split video into 10 frames
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_interval = frame_count // 10  # Choose 10 frames evenly spaced
frames = []

for i in range(10):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
    ret, frame = cap.read()
    if ret:
        frames.append(frame)
    else:
        print(f"Could not read frame {i}")
        break

cap.release()

# Step 3 & 4: Apply MTCNN and save cropped face images
for idx, frame in enumerate(frames):
    result = detector.detect_faces(frame)
    if result:
        # Assume only one face per frame for simplicity, using the first detected face
        x, y, width, height = result[0]['box']
        cropped_face = frame[y:y+height, x:x+width]
        save_path = os.path.join(output_folder, f'cropped_face_{idx}.jpg')
        cv2.imwrite(save_path, cropped_face)
    else:
        print(f"No face detected in frame {idx}")

print("Cropped face images saved successfully.")


1/1 [==============================] - 0s 30ms/step
Cropped face images saved successfully.


In [16]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import Dropout

# Define the FixedDropout layer
class FixedDropout(Dropout):
    def __init__(self, rate, **kwargs):
        super(FixedDropout, self).__init__(rate, **kwargs)
        self.supports_masking = True

    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return None
        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                       for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)

# Step 1: Load the model with custom objects
model = load_model('balan_finallllll_model.h5', custom_objects={'FixedDropout': FixedDropout})

# Folder containing cropped images
cropped_images_folder = 'cropped_images'

# Step 2: Preprocess the images (input size is 128x128)
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Adjusting input size to 128x128
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize to [0, 1] range
    return img

# Step 3: Pass images through the model
predictions = []
for image_file in os.listdir(cropped_images_folder):
    image_path = os.path.join(cropped_images_folder, image_file)
    img = preprocess_image(image_path)
    pred = model.predict(img)
    predictions.append(pred)

# Step 4: Average the predictions
average_prediction = np.mean(predictions)

# Step 5: Classify as fake or real
threshold = 0.2  # Assuming a binary classification with sigmoid output
if average_prediction > threshold:
    classification = 'Real'
else:
    classification = 'Fake'

print(f"The average prediction is: {average_prediction:.4f}")
print(f"The images are classified as: {classification}")


1/1 [==============================] - 0s 35ms/step
The average prediction is: 0.2803
The images are classified as: Real
